In [50]:
from utils import generate_array
from graphics import display_grid
from grid import Grid
import numpy as np

In [51]:
train_xs = np.load("grids_data.npy")
train_ys = np.load("actions_data.npy")
print(train_xs.shape, train_ys.shape)


(918, 5, 5) (918,)


In [52]:
action_dict = {(0, 0):0, (-1, 0):1, (0, 1):2, (1, 0):3, (0, -1):4}

In [3]:
"""
train_xs = []
train_ys = []

# makes 5x5 test grid and chooses random action until terminal state is reached
grid = Grid(5,random=True)
grid.agent_pos
#display_grid(grid)
#print(grid.current_state)
while not grid.terminal_state:
    print("")
    print(generate_array(grid))
    train_xs.append(generate_array(grid))
    action =  tuple(grid.legal_actions())[0]#random.choice(tuple(grid.legal_actions()))
    print(action)
    train_ys.append(action_dict[action])
    
    grid.T(action)
    #display_grid(grid)
    #print(grid.current_state)
"""


[[3 0 0 0 0]
 [1 0 4 2 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
(0, 0)

[[0 3 0 0 0]
 [1 0 4 2 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
(0, 0)

[[0 0 3 0 0]
 [1 0 4 2 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
(0, 0)


In [4]:
"""
train_xs = np.array(train_xs, dtype=np.int64)
train_ys = np.array(train_ys, dtype=np.int64)
print(train_xs.shape, train_ys.shape)
"""

(3, 5, 5) (3,)


In [53]:
C = int(np.max(train_xs))+1
print(C)

5


In [54]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [55]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(C, 10, 5, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(10, 30, 3, padding=1)
        self.fc1 = nn.Linear(30 * 1 * 1, 5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 30 * 1 * 1)
        x = self.fc1(x)
        return x
    
net = Net()

In [56]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# input shape b x c x h x w to net
# so need to unsqueeze to make channel dimension of 1, could also make each agent it's own channel instead
B, H, W = train_xs.shape
train_xs = torch.from_numpy(train_xs).unsqueeze(1)
onehot_train_xs = torch.zeros(B, C, H, W)

print(train_xs.shape, onehot_train_xs.shape)
onehot_train_xs.scatter_(1, train_xs, torch.ones(onehot_train_xs.shape))
print(onehot_train_xs[0])
train_ys = torch.from_numpy(train_ys)

torch.Size([918, 1, 5, 5]) torch.Size([918, 5, 5, 5])
tensor([[[1., 1., 1., 0., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 0., 1.],
         [1., 1., 1., 1., 0.],
         [1., 1., 0., 1., 1.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0.]]])


In [57]:
for epoch in range(300):  # loop over the dataset multiple times

    running_loss = 0.0
    
    inputs, labels = onehot_train_xs, train_ys

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss += loss.item()
    if epoch%10==9:
        print('Epoch:{}, loss: {:.3f}'.format(epoch + 1, running_loss))
        running_loss = 0.0

Epoch:10, loss: 1.596
Epoch:20, loss: 1.542
Epoch:30, loss: 1.484
Epoch:40, loss: 1.427
Epoch:50, loss: 1.370
Epoch:60, loss: 1.313
Epoch:70, loss: 1.255
Epoch:80, loss: 1.198
Epoch:90, loss: 1.144
Epoch:100, loss: 1.100
Epoch:110, loss: 1.070
Epoch:120, loss: 1.054
Epoch:130, loss: 1.047
Epoch:140, loss: 1.045
Epoch:150, loss: 1.044
Epoch:160, loss: 1.044
Epoch:170, loss: 1.044
Epoch:180, loss: 1.044
Epoch:190, loss: 1.043
Epoch:200, loss: 1.043
Epoch:210, loss: 1.043
Epoch:220, loss: 1.043
Epoch:230, loss: 1.043
Epoch:240, loss: 1.043
Epoch:250, loss: 1.042
Epoch:260, loss: 1.042
Epoch:270, loss: 1.042
Epoch:280, loss: 1.042
Epoch:290, loss: 1.042
Epoch:300, loss: 1.042


In [61]:
onehot_train_xs = torch.zeros(B, C, H, W)
test = np.array([[0, 2, 0, 0, 1],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 3],
        [0, 0, 0, 4, 0]])
onehot_train_xs.scatter_(1, test, torch.ones(onehot_train_xs.shape))
outputs = net(test)

TypeError: scatter_() received an invalid combination of arguments - got (int, numpy.ndarray, Tensor), but expected one of:
 * (int dim, Tensor index, Tensor src)
      didn't match because some of the arguments have invalid types: ([32;1mint[0m, [31;1mnumpy.ndarray[0m, [32;1mTensor[0m)
 * (int dim, Tensor index, Number value)
      didn't match because some of the arguments have invalid types: ([32;1mint[0m, [31;1mnumpy.ndarray[0m, [31;1mTensor[0m)


In [60]:
print(onehot_train_xs[0])

tensor([[[1., 1., 1., 0., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 0., 1.],
         [1., 1., 1., 1., 0.],
         [1., 1., 0., 1., 1.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0.]]])
